## 수익률 표준편차

In [2]:
import pandas as pd

# cp949 또는 euc-kr 인코딩 시도
df = pd.read_csv('../data/주가수익률_표준편차.csv', encoding='cp949')
df.drop("Unnamed: 0", axis=1, inplace=True)
# 만약 cp949에서 에러가 발생하면 utf-8로 시도해보세요
# df = pd.read_csv('data/주가수익률_표준편차.csv', encoding='utf-8')
df.to_csv("주가수익률test.csv", encoding='utf-8', index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../data/주가수익률_표준편차.csv'

In [30]:
df = df[df['기간구간'] != '기타']
df

,회사명,공모가 (원),회계년도,종가(원),수익률,일자순번,기간구간,수익률_표준편차
0,AJ네트웍스,34300,2015-08-21,35500,0.0350,1,1~5일,0.089380
1,AJ네트웍스,34300,2015-08-24,32400,-0.0873,2,1~5일,0.089380
2,AJ네트웍스,34300,2015-08-25,31100,-0.0401,3,1~5일,0.089380
3,AJ네트웍스,34300,2015-08-26,35450,0.1399,4,1~5일,0.089380
4,AJ네트웍스,34300,2015-08-27,37850,0.0677,5,1~5일,0.089380
...,...,...,...,...,...,...,...,...
741814,힘스,20000,2017-08-10,27100,-0.0356,16,16~20일,0.020914
741815,힘스,20000,2017-08-11,26800,-0.0111,17,16~20일,0.020914
741816,힘스,20000,2017-08-14,26800,0.0000,18,16~20일,0.020914
741817,힘스,20000,2017-08-16,27400,0.0224,19,16~20일,0.020914


In [33]:
df[df['기간구간'] != '기타'].to_csv("주가수익률test.csv", encoding='utf-8', index=False)

In [15]:
rename_map = {
    "1~5일": "1~5일 수익률 표준편차",
    "6~10일": "6~10일 수익률 표준편차",
    "11~15일": "11~15일 수익률 표준편차",
    "16~20일": "16~20일 수익률 표준편차",
}

pivot = (
    df.pivot_table(index="회사명",
                   columns="기간구간",
                   values="수익률_표준편차",
                   aggfunc="mean")          # 중복 → 평균
      .rename(columns=rename_map)
)

ordered_cols = list(rename_map.values())
wide_df = pivot[ordered_cols].reset_index()

# 저장
wide_csv_path = "주가수익률test_WIDE.csv"
wide_df.to_csv(wide_csv_path, index=False, encoding="utf-8-sig")

print("CSV 파일 저장 위치:")
print(" - 변환본 :", wide_csv_path)

CSV 파일 저장 위치:
 - 변환본 : 주가수익률test_WIDE.csv


In [19]:
df = pd.read_csv("../data/주가수익률_final.csv", index_col=0)
df.isna().sum()

회사명                0
1~5일 수익률 표준편차      0
6~10일 수익률 표준편차     2
11~15일 수익률 표준편차    6
16~20일 수익률 표준편차    6
stock_code         0
dtype: int64

In [8]:
import pandas as pd

gi = pd.read_csv("../data/gi.csv")
gi.drop("Unnamed: 0", axis=1, inplace=True)
df = pd.read_csv("../data/주가수익률_final.csv", index_col=0)

filtered_df = df[df['회사명'].isin(gi['회사명'])]

print(len(filtered_df))
print(filtered_df.reset_index(drop=True).to_csv("../data/주가수익률_filtered.csv"))

533
None


In [ ]:
import pandas as pd

csi = pd.read_csv("../data/경기국면/경기국면(CSI).csv")
merged_df = pd.merge(filtered_df, csi[['회사명', '경기국면']], on='회사명', how='left')

# 결과 확인
merged_df.to_csv("../data/주가수익률_CSI.csv", index=False)


In [16]:
import pandas as pd

동행 = pd.read_csv("../data/경기국면/경기국면(경기동행).csv")
filtered_df = pd.read_csv("../data/주가수익률/주가수익률_filtered.csv", index_col=0)
merged_df = pd.merge(filtered_df, 동행[['회사명', '경기국면']], on='회사명', how='left')

# 결과 확인
merged_df.to_csv("../data/주가수익률_경기동행.csv", index=False)


In [30]:
import pandas as pd

pre = pd.read_csv("../data/pre_data.csv")
pre.drop("Unnamed: 0", axis=1, inplace=True)

filtered_df = pd.read_csv("../data/주가수익률/주가수익률_filtered.csv", index_col=0)
merged_df = pd.merge(filtered_df, pre, on='stock_code', how='left')

merged_df.drop("Unnamed: 0.1", axis=1, inplace=True)
merged_df['회사명'] = merged_df['회사명_x'].combine_first(merged_df['회사명_y'])
merged_df.drop("회사명_y", axis=1, inplace=True)
merged_df.drop("회사명_x", axis=1, inplace=True)

동행 = pd.read_csv("../data/경기국면/경기국면(경기동행).csv")
merged_df = pd.merge(merged_df, 동행[['회사명', '경기국면']], on='회사명', how='left')

print(merged_df.isna().sum())

# merged_df.to_csv("../data/주가수익률/주가수익률_temp.csv", index=False)

ParserError: Error tokenizing data. C error: Expected 11 fields in line 537, saw 78


In [ ]:
import pandas as pd

pre = pd.read_csv("../data/pre_data.csv")
# pre.drop(["Unnamed: 0", "Unnamed: 0.1", "WICS_large", "target_1d", "target_2d",
#          "target_7d", "target_15d", "target_1m", "target_3m", "target_6m",
#           "회사명_d1", "업종", "상장일", "유동자산", "유동부채", "부채총계",
#           "자본금", "자본총계", "매출액", "영업이익", "당기순이익", "회사명_bs",
#           "z_simplified", "corp_code", "kospi", "call", "modified_esi", "공모금액 (천원)",
#           "공모가 (원)", "상장일_ipo", "최초상장주식수 (주)", "상장주선인",
#           "log_TA", "log_S", "TA", "WC", "capital_deficit","회사명_d1", "6개월 확약",
#           "3개월 확약", "1개월 확약", "15일 확약", "미확약", "수요예측_최종값", "회사명_bs",
#           "업력", ""], axis=1, inplace=True)
pre.drop(["Unnamed: 0", "Unnamed: 0.1", "WICS_large", "target_1d", "target_2d",
         "target_7d", "target_15d", "target_1m", "target_3m", "target_6m",
          "회사명_d1", "업종", "유동자산", "유동부채", "부채총계",
          "자본금", "자본총계", "매출액", "영업이익", "당기순이익", "회사명_bs",
          "z_simplified", "corp_code", "kospi", "call", "modified_esi", "공모금액 (천원)",
          "공모가 (원)", "상장일_ipo", "최초상장주식수 (주)", "상장주선인",
          "log_TA", "log_S", "TA", "WC", "capital_deficit", "wc_neg_dummy", "roe_nan", "ta_negative",
          "current_ratio", "wc_sales", "wc_ta","cl_tl", "de_ratio", "equity_ratio","cs_te",
          "debt_asset", 'op_margin', 'net_margin', 'roe', 'roe_sign_dummy',
          'roe_nan', 'roa', 'oi_ta', 'ni_oi_ratio', 'asset_turnover',
          'sales_equity', "회사명_d1", "6개월 확약", "3개월 확약", "1개월 확약", "15일 확약", "미확약",
          "수요예측_최종값", "회사명_bs", "업력", "우리사주조합", "기관투자자", "일반투자자"], axis=1, inplace=True)          

# pre.to_csv("../data/macro_data.csv", index=False)

In [109]:
import pandas as pd

macro = pd.read_csv("../data/macro_data.csv")
fileter = pd.read_csv("../data/주가수익률/주가수익률_filtered.csv")

merged = pd.merge(fileter, macro, on="stock_code", how="left")
merged["회사명"] = merged["회사명_x"].combine_first(merged["회사명_y"])
merged.drop(["Unnamed: 0", "회사명_y", "회사명_x"], axis=1, inplace=True)
merged.to_csv("../data/주가수익률/주가수익률_base.csv", index=False)


In [104]:
import pandas as pd

df = pd.read_csv("../data/주가수익률/주가수익률_base.csv")
df.drop("Unnamed: 0", axis=1, inplace=True)
df.columns.tolist()

# df.to_csv("../data/주가수익률/주가수익률_base.csv", encoding="utf-8-sig", index=True)

['1~5일 수익률 표준편차',
 '6~10일 수익률 표준편차',
 '11~15일 수익률 표준편차',
 '16~20일 수익률 표준편차',
 'stock_code',
 'kospi200(-20)',
 'nasdaq(-20)',
 'putcall(-20)',
 'VIXCLS',
 '미국10년물국채금리수익률(-20)',
 '시장금리국고채3년',
 '환율변동률(-20)',
 '코스피변동률(-20)',
 '코스닥변동률(-20)',
 '6개월 확약',
 '3개월 확약',
 '1개월 확약',
 '15일 확약',
 '미확약',
 '수요예측_최종값',
 '업력',
 'wc_neg_dummy',
 'ta_negative',
 'current_ratio',
 'wc_sales',
 'wc_ta',
 'cl_tl',
 'de_ratio',
 'equity_ratio',
 'cs_te',
 'debt_asset',
 'op_margin',
 'net_margin',
 'roe',
 'roe_sign_dummy',
 'roe_nan',
 'roa',
 'oi_ta',
 'ni_oi_ratio',
 'asset_turnover',
 'sales_equity',
 '우리사주조합',
 '기관투자자',
 '일반투자자',
 '회사명',
 '런던_가스오일(-1~-5)',
 '런던_가스오일(-6~-10)',
 '런던_가스오일(-11~-15)',
 '런던_가스오일(-16~-20)',
 '난방유(-1~-5)',
 '난방유(-6~-10)',
 '난방유(-11~-15)',
 '난방유(-16~-20)',
 '탄소배출권(-1~-5)',
 '탄소배출권(-6~-10)',
 '탄소배출권(-11~-15)',
 '탄소배출권(-16~-20)',
 '브렌트유(-1~-5)',
 '브렌트유(-6~-10)',
 '브렌트유(-11~-15)',
 

In [111]:
import pandas as pd

df_base = pd.read_csv("../data/주가수익률/주가수익률_base.csv")
df_base.columns.tolist()

['1~5일 수익률 표준편차',
 '6~10일 수익률 표준편차',
 '11~15일 수익률 표준편차',
 '16~20일 수익률 표준편차',
 'stock_code',
 '상장일',
 'kospi200(-20)',
 'nasdaq(-20)',
 'putcall(-20)',
 'VIXCLS',
 '미국10년물국채금리수익률(-20)',
 '시장금리국고채3년',
 '환율변동률(-20)',
 '코스피변동률(-20)',
 '코스닥변동률(-20)',
 '런던_가스오일(-1~-5)',
 '런던_가스오일(-6~-10)',
 '런던_가스오일(-11~-15)',
 '런던_가스오일(-16~-20)',
 '난방유(-1~-5)',
 '난방유(-6~-10)',
 '난방유(-11~-15)',
 '난방유(-16~-20)',
 '탄소배출권(-1~-5)',
 '탄소배출권(-6~-10)',
 '탄소배출권(-11~-15)',
 '탄소배출권(-16~-20)',
 '브렌트유(-1~-5)',
 '브렌트유(-6~-10)',
 '브렌트유(-11~-15)',
 '브렌트유(-16~-20)',
 '가솔린(-1~-5)',
 '가솔린(-6~-10)',
 '가솔린(-11~-15)',
 '가솔린(-16~-20)',
 '천연가스(-1~-5)',
 '천연가스(-6~-10)',
 '천연가스(-11~-15)',
 '천연가스(-16~-20)',
 'WTI유(-1~-5)',
 'WTI유(-6~-10)',
 'WTI유(-11~-15)',
 'WTI유(-16~-20)',
 '금(-1~-5)',
 '금(-6~-10)',
 '금(-11~-15)',
 '금(-16~-20)',
 '은(-1~-5)',
 '은(-6~-10)',
 '은(-11~-